In [24]:
import numpy as np
import cv2
import time

img = cv2.imread('scaled/field.png')
img = cv2.imread("scaled/IMG_1133.jpg")
img = cv2.imread("scaled/scaleImage2.jpg")

start = time.time()

# resize img for speed
height, width = img.shape[:2]
img = cv2.resize(img, (round(width / 5), round(height / 5)), interpolation=cv2.INTER_AREA)

Z = img.reshape((-1,3))

# convert to np.float32
Z = np.float32(Z)

# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 2
ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))

print(time.time() - start)

display(res2)
cv2.waitKey(0)
cv2.destroyAllWindows()

0.29399800300598145
0.31499361991882324


In [3]:
import cv2


def resizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    """ return resized image """
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)


def display(img) -> None:
    """ display 800px wide image and wait for enter """
    img = resizeWithAspectRatio(img, width=800)
    cv2.imshow("", img)
    cv2.waitKey(0)